In [1]:
%load_ext autoreload
%autoreload 2
%pwd
import hdf5plugin

import matplotlib.pyplot as plt
from diffractem import io, tools, pre_process, version
from diffractem.stream_parser import StreamParser, chop_stream
from diffractem.dataset import Dataset
import numpy as np
import pandas as pd
import os
import matplotlib
from glob import glob

bin_path = '/opts/crystfel_eminus/bin/' # might be different than standard
cfver = !{bin_path + 'partialator'} --version
print(f'Running on diffractem:', version())
print(f'Running on', cfver[0])
print(f'Current path is:', os.getcwd())

Running on diffractem: v0.2.1-52-g0d34dce
Running on CrystFEL: 0.8.0+f9101682
Current path is: /nas/processing/serialed/GV/publication


# Merging of serial data sets
...from stream files, using partialator and ambigator from CrystFEL. Handles parallel processing of merging runs with different parameters and/or input stream files, as well as creation of custom-split files.

Also includes calculation, export and visualization of shell data.

### Preparation of a command script for partialator
The next section prepares command strings comprised of NaN removal from stream files, ambigator, and partialator. Often, you will want to run partialator (which does not parallelize well) for many settings streams in parallel, hence the loop(s). The command strings are written into a shell script (patialator_run.sh), including terminating ampersands for parallel execution. Run this in an external terminal (it won't work with a Jupyter cell magic).

In this example, we will try to run it with three settings for `push-res`, and also compare the stream files derived from the initial aggregated images, and the ones with optimized integration.

In [3]:
!echo "#!/bin/sh" > partialator_run.sh
stream_list = ['stream/frame0to4.stream', 'stream/aggregated.stream']
os.makedirs('merged', exist_ok=True)

for pr in [1, 2, 3]:
    for stream_name in stream_list:
        
        # change these parameters to control the merging ---
        original = stream_name
        partialator_out = 'merged/' + original.rsplit('/', 1)[-1].rsplit('.', 1)[0] + \
            f'_{pr:.2f}'.replace('.','_') + f'.hkl' # output filename
        point_group = 'm-3'
        apparent_symmetry = 'm-3m' # see ambigator documentation
        fix_nans = True # required if some peak integrations went wrong
        procs = 40      # number of processes (rather hypothetical)
        it = 3          # number of scaling iterations
        split = False   # use custom-split?
        min_meas = 10   # minimum number of peak observations to count as observed
        # find many more settings below in call_partialator
        # ----
        
        callstr = '('
        
        if fix_nans: 
            ambigator_in = original.rsplit('.', 1)[0] + '_nonan.stream'           
            callstr += f'sed /nan/d {original} > {ambigator_in}; '
        else:
            ambigator_in = original
            
        if (point_group != apparent_symmetry) and point_group is not None:
            ambigator_out = original.rsplit('.', 1)[0] + '_amb.stream'
            amstr = tools.make_command(bin_path + 'ambigator', ambigator_in, 
                               {'o': ambigator_out, 'y': point_group, 
                                'w': apparent_symmetry, 'j': procs}, 
                               highres=2.5, lowres=10, ncorr=1000)
            callstr += f'{amstr}; '
        else:
            ambigator_out = ambigator_in

        partstr = tools.call_partialator(ambigator_out, point_group, partialator_out, 
                                         opts={'no-polarisation': True, 
                                               'no-Bscale': False, 'no-scale': False,
                                               'force-radius': 0.01,
                                               'force-bandwidth': 0.0005, 'push-res': pr,
                                               'custom-split': original.rsplit('.',1)[0] \
                                               + '_frames.txt' if split else False,
                                               'min-measurements': min_meas
                                              }, procs=procs,
                                         exc= bin_path + 'partialator', iterations=it)
        
        callstr += f'{partstr}) &'

        print(callstr)
        !echo "{callstr}" >> partialator_run.sh        

(sed /nan/d stream/frame0to4.stream > stream/frame0to4_nonan.stream; /opts/crystfel_eminus/bin/ambigator stream/frame0to4_nonan.stream -o stream/frame0to4_amb.stream -y m-3 -w m-3m -j 40 --highres=2.5 --lowres=10 --ncorr=1000; /opts/crystfel_eminus/bin/partialator -y m-3 -i stream/frame0to4_amb.stream -o merged/frame0to4_1_00.hkl -j 40 -n 3 -m unity --no-polarisation --force-radius=0.01 --force-bandwidth=0.0005 --push-res=1 --min-measurements=10) &
(sed /nan/d stream/aggregated.stream > stream/aggregated_nonan.stream; /opts/crystfel_eminus/bin/ambigator stream/aggregated_nonan.stream -o stream/aggregated_amb.stream -y m-3 -w m-3m -j 40 --highres=2.5 --lowres=10 --ncorr=1000; /opts/crystfel_eminus/bin/partialator -y m-3 -i stream/aggregated_amb.stream -o merged/aggregated_1_00.hkl -j 40 -n 3 -m unity --no-polarisation --force-radius=0.01 --force-bandwidth=0.0005 --push-res=1 --min-measurements=10) &
(sed /nan/d stream/frame0to4.stream > stream/frame0to4_nonan.stream; /opts/crystfel_emin

### Analysis of shell data
Theses cells calculate per-shell and overall merging statistics, using CrystFELs _check_hkl_ and _compare_hkl_. Returns pandas DataFrames for overall and shell statistics.

In [4]:
os.makedirs('shell', exist_ok=True)

# get a list of all hkl files
hklfiles = sorted(glob('merged/aggregated_?_??.hkl'))
hklfiles.extend(sorted(glob('merged/frame0to4_?_??.hkl')))

# set up the shells
lowres = 72.973
highres = 1.55
nshells = 10

# load the cell file you used, and set the point group
cell = 'GV.cell'
point_group = 'm-3'

# chose figures of merit for compare_hkl (see help)
foms = ['CC', 'CCstar', 'Rsplit']
    
def analyze_hkl(fn):
    fnroot = fn.rsplit('/',1)[-1].rsplit('.')[0]
    print('Analyzing ' + fnroot + '...')
    try:
        callstr = tools.make_command(bin_path + 'check_hkl', fn, 
                                     {'y': point_group, 'p': cell},
                                      {'shell-file': f'shell/hkl_{fnroot}.dat'}, 
                                     nshells=nshells, lowres=lowres, highres=highres)
        so = !{callstr}

        sd1 = pd.read_csv(f'shell/hkl_{fnroot}.dat', delim_whitespace=True, header=None, skiprows=1)
        sd1.columns = columns=['Center 1/nm', 'nref', 'Possible',  
                                        'Compl', 'Meas', 'Red', 'SNR', 'Std dev', 
                                        'Mean', 'd/a', 'Min 1/nm', 'Max 1/nm']

        for idx, fom in enumerate(foms):
            callstr = tools.make_command(bin_path + 'compare_hkl', [fn + '1', fn + '2'], 
                                     {'y': point_group, 'p': cell},
                                      {'shell-file': f'shell/cmp_{fnroot}.dat', 'sigma-cutoff': -4}, 
                                         nshells=nshells, lowres=lowres, highres=highres, fom=fom)

            so2 = !{callstr}
            so += so2

            if idx==0:
                sd2 = pd.read_csv(f'shell/cmp_{fnroot}.dat', delim_whitespace=True, 
                                     header=None, skiprows=1)
                sd2 = sd2[[0, 2, 3, 4, 5, 1]]
            else:
                sd2 = pd.concat([sd2, pd.read_csv(f'shell/cmp_{fnroot}.dat', delim_whitespace=True, 
                                     header=None, skiprows=1, usecols=[1])], axis=1, ignore_index=True)

        sd2.columns = ['Center 1/nm', 'nref', 'd/A', 'Min 1/n', 'Max 1/nm'] + foms

        sd = pd.merge(sd1, sd2, on='Center 1/nm', suffixes=('', '__2'))
        sd.drop([c for c in sd.columns if c.endswith('__2')], axis=1, inplace=True)
        sd['hklfile'] = fn
    except Exception as err:
        print('Error during analyzing file', fn)
        raise err
    
    return fn, sd, so

from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor() as exc:
    out = exc.map(analyze_hkl, hklfiles)

out = list(out)

# output data mangling
stdout = {s[0]: s[2] for s in out}
sd = pd.concat([s[1] for s in out], axis=0, ignore_index=True)

overall = {}
for fn, lst in stdout.items():
    vals = {}
    for l in [l for l in lst if ('overall' in l.lower()) and ('=' in l)]:
        sect = l.split('=')
        fld = sect[0].strip().rsplit(' ', 1)[-1]
        #print(l)
        vals[fld] = float(sect[1].strip().split(' ')[0])
    overall[fn] = vals
overall = pd.DataFrame(overall).T

Analyzing aggregated_1_00...
Analyzing aggregated_2_00...
Analyzing aggregated_3_00...
Analyzing frame0to4_1_00...
Analyzing frame0to4_2_00...
Analyzing frame0to4_3_00...


In [5]:
# give friendly names datasets... in this case we just assign IDs...
idnums = {fn: ident for ident, fn in zip(range(len(overall)),sorted(overall.index.values))}
sd['identifier'] = sd.hklfile.replace(idnums)
overall['identifier'] = overall.index.to_series().replace(idnums)

### Print overall and per-shell statistics
Overall: just show DataFrame
Per-shell: use pivot table to display a particular FOM, like CC.

In this case, we'll see that the re-integration makes quite some difference, especially in terms of the CC drop-off at high resolutions. push-res=2 (it's 1/nm, actually) helps to squeeze out high-resolution data, which still have an acceptable CC and would be discarded by a more conservative resolution estimation. push-res=3 does not make a significant difrence.

In [6]:
overall

,<snr>,CC,CC*,Rsplit,completeness,redundancy,identifier
merged/aggregated_1_00.hkl,5.806275,0.996842,0.999209,11.57,63.488345,67.788847,0
merged/aggregated_2_00.hkl,5.034441,0.993877,0.998463,12.96,78.480089,62.034986,1
merged/aggregated_3_00.hkl,5.031179,0.993873,0.998462,12.98,78.545371,62.020876,2
merged/frame0to4_1_00.hkl,5.478837,0.990242,0.997546,11.81,65.462156,72.186954,3
merged/frame0to4_2_00.hkl,4.539161,0.990040,0.997494,13.38,81.532967,66.227157,4
merged/frame0to4_3_00.hkl,4.532576,0.990044,0.997495,13.39,81.655850,66.180822,5


In [7]:
sd.pivot(index='d/A', columns='identifier', values=['CC','Compl']).sort_index(ascending=False)

CC                                                    \
identifier         0         1         2         3         4         5   
d/A                                                                      
6.39        0.997389  0.994392  0.994390  0.990068  0.990168  0.990169   
2.96        0.962490  0.962677  0.962679  0.969382  0.969146  0.969146   
2.47        0.858409  0.844447  0.844450  0.908125  0.902326  0.902390   
2.20        0.792263  0.771637  0.771465  0.873670  0.856109  0.856087   
2.03        0.755510  0.713491  0.712252  0.858776  0.825072  0.823978   
1.89        0.566242  0.495958  0.498973  0.741910  0.698647  0.696229   
1.79        0.496555  0.358398  0.357416  0.606059  0.521002  0.523039   
1.71             NaN  0.348273  0.338661  0.604582  0.401499  0.403964   
1.64             NaN  0.307089  0.303675       NaN  0.201995  0.201892   
1.58             NaN  0.828246  0.828248       NaN  0.413056  0.413035   

             Compl                                          
identifier       0       1       2       3       4       5  
d/A                                                         
6.39        100.00  100.00  100.00  100.00  100.00  100.00  
2.96        100.00  100.00  100.00  100.00  100.00  100.00  
2.47        100.00  100.00  100.00   99.96   99.96   99.96  
2.20         99.69   99.96   99.96   99.85   99.96   99.96  
2.03         97.48   99.81   99.81   98.03   99.96   99.96  
1.89         84.18   98.31   98.31   88.83   98.81   98.81  
1.79         45.77   90.81   90.92   56.87   94.45   94.49  
1.71          4.42   61.58   61.89    7.92   70.20   70.66  
1.64          0.00   24.77   24.92    0.00   35.19   35.66  
1.58          0.00    7.59    7.67    0.00   15.10   15.38

### Export
...to CSV, and Excel (the latter only works with some packages installed)

In [8]:
lbl = '10_shell_data'
sd.to_csv(f'shell/{lbl}.csv')

In [9]:
# Export to Excel file
with pd.ExcelWriter(f'shell/{lbl}.xlsx') as writer:
    for ii, grp in sd.groupby('identifier'):
        grp.to_excel(writer, sheet_name=str(ii))

In [10]:
# re-load if you want (for later re-execution)
lbl = '10_shell_data'
sd = pd.read_csv('shell/' + lbl + '.csv', index_col=0)

### Plot
...ideally using an interactive backend, like `qt`, `notebook` (for Jupyter notebook) or `ipympl` (for Jupyter lab). You can choose which 4 FOMs to plot as a function of resolution, and their display ranges. From the $CC=1/7$, or $CC^*=1/2$ criterion, we can read off that a good cut-off would be at around 1.7 Angstrom, though completeness gets already low-ish around there.

In our serial electron protein diffraction paper, with 10x the shots, we pushed it to 1.55.

Note that the apparently higher Mean signal and SNR for the initial aggregated sets (0-2) are an artefact arising from CrystFEL's background gradient correction, which is switched off in the re-integrated sets.

In [11]:
%matplotlib ipympl

fh, axs = plt.subplots(2,2,figsize=(18/2.54,22/2.54),dpi=120,sharex=True)
axs = axs.ravel()

# pick your FOMs and their y ranges
FOMs = [('CC', 0, 1), ('Mean', 0, 20), ('SNR', 0, 20), ('Compl', 0, 100)]

for (fn, ident), grp in sd.groupby(['hklfile','identifier']):
    
    if ident >= 20:
        ls = ':'
    else:
        ls = '-'

    lbl = ident
    for ax, (fom, xmin, xmax) in zip(axs, FOMs):
        ax.plot(grp['Center 1/nm'], grp[fom], label=lbl, ls=ls)
        ax.set_title(fom)
        ax.set_ylim((xmin, xmax))
        ax.grid(True)
        if fom in ['CC', 'CCstar']:
            ax.axhline(0.143 if fom == 'CC' else 0.5,ls=':')
        
axs[-1].legend(ncol=2, fontsize='xx-small')
axs[-1].set_xlabel(r'Resolution shell/nm$^{-1}$')

if 'nbagg' in matplotlib.get_backend(): # hack for ipympl backend
    fh.canvas.layout.height = f'{fh.get_size_inches()[1]}in'
fh.subplots_adjust(wspace=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Congratulations
Now you have hopefully picked an optimized `.hkl` file, which you can load into your favorite phasing software!